# Meshes

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from ngsolve import *
import matplotlib.pyplot as plt
import sys
import gc

sys.path.insert(1, "/Users/gpin/Repos/EVErrorEst/scripts")

from main_utils import *
from adapt_lshape import *
from error import *
from ngsolve.webgui import Draw
from IPython.display import display, HTML

In [2]:
def reset_mesh():
    return Mesh(make_l_shape().GenerateMesh(maxh=0.3))


def error_estimator_landscape(gf):
    """
    Compute the landscape error estimator.
    """
    h = specialcf.mesh_size
    n = specialcf.normal(gf.space.mesh.dim)
    xs = [x, y, z][: gf.space.mesh.dim]

    grad_gf = grad(gf)
    div_grad_gf = sum((grad_gf[i].Diff(xs[i]) for i in range(gf.space.mesh.dim)))

    integrand_1 = -div_grad_gf - 1.0
    integrand_2 = (grad_gf - grad_gf.Other()) * n

    eta_1 = Integrate(
        h**2 * InnerProduct(integrand_1, integrand_1) * dx,
        gf.space.mesh,
        element_wise=True,
    )
    eta_2 = Integrate(
        0.5 * h * InnerProduct(integrand_2, integrand_2) * dx(element_boundary=True),
        gf.space.mesh,
        element_wise=True,
    )

    eta = np.sqrt(eta_1.NumPy().real + eta_2.NumPy().real)
    etas = {
        "eta_1": eta_1.NumPy().real,
        "eta_2": eta_2.NumPy().real,
    }

    return eta, etas


def error_estimator_ev(gf, ev):
    """
    Compute the landscape error estimator.
    """
    h = specialcf.mesh_size
    n = specialcf.normal(gf.space.mesh.dim)
    xs = [x, y, z][: gf.space.mesh.dim]

    grad_gf = grad(gf)
    div_grad_gf = sum((grad_gf[i].Diff(xs[i]) for i in range(gf.space.mesh.dim)))

    integrand_1 = -div_grad_gf - ev * gf
    integrand_2 = (grad_gf - grad_gf.Other()) * n

    eta_1 = Integrate(
        h**2 * InnerProduct(integrand_1, integrand_1) * dx,
        gf.space.mesh,
        element_wise=True,
    )
    eta_2 = Integrate(
        0.5 * h * InnerProduct(integrand_2, integrand_2) * dx(element_boundary=True),
        gf.space.mesh,
        element_wise=True,
    )

    eta = np.sqrt(eta_1.NumPy().real + eta_2.NumPy().real)
    etas = {
        "eta_1": eta_1.NumPy().real,
        "eta_2": eta_2.NumPy().real,
    }

    return eta, etas

In [8]:
# FES parameters
ORDER = 1
IS_COMPLEX = True

# Adaptivity parameters
MAX_ITER = 10
MAX_NDOFS = 300_000
# THETA = 0.95

# FEAST parameters
NSPAN = 6
CHECKS = False
RADIUS = 2.0

#### High tolerance

In [9]:
mesh = reset_mesh()
fes = H1(mesh, order=ORDER, complex=IS_COMPLEX, dirichlet="boundary", autoupdate=True)

u, v = fes.TnT()

a = BilinearForm(fes)
a += grad(u) * grad(v) * dx

m = BilinearForm(fes)
m += u * v * dx

f = LinearForm(fes)
f += 1.0 * v * dx

assemble(a, m, f)

In [10]:
THETA = 0.95

iteration = 0
ndofs = 0
sol = GridFunction(fes, name="Landscape", autoupdate=True)

# Main loop
Draw(mesh)
while iteration < MAX_ITER and ndofs < MAX_NDOFS:
    # Update
    assemble(a, f)
    ndofs = fes.ndof
    iteration += 1
    # Solve
    sol.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
    # Estimate
    eta, _ = error_estimator_landscape(sol)
    # Mark
    max_eta = max(eta)
    mesh.ngmesh.Elements2D().NumPy()["refine"] = eta > THETA * max_eta
    # Refine
    mesh.Refine()
    Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

...

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

#### Low tolerance

In [11]:
mesh = reset_mesh()
fes = H1(mesh, order=ORDER, complex=IS_COMPLEX, dirichlet="boundary", autoupdate=True)

u, v = fes.TnT()

a = BilinearForm(fes)
a += grad(u) * grad(v) * dx

m = BilinearForm(fes)
m += u * v * dx

f = LinearForm(fes)
f += 1.0 * v * dx

assemble(a, m, f)

In [12]:
THETA = 0.5

iteration = 0
ndofs = 0
sol = GridFunction(fes, name="Landscape", autoupdate=True)

# Main loop
Draw(mesh)
while iteration < MAX_ITER and ndofs < MAX_NDOFS:
    # Update
    assemble(a, f)
    ndofs = fes.ndof
    iteration += 1
    # Solve
    sol.vec.data = a.mat.Inverse(fes.FreeDofs()) * f.vec
    # Estimate
    eta, _ = error_estimator_landscape(sol)
    # Mark
    max_eta = max(eta)
    mesh.ngmesh.Elements2D().NumPy()["refine"] = eta > THETA * max_eta
    # Refine
    mesh.Refine()
    Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

...

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…